## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [ ]:
!nvidia-smi

Sun Oct 22 13:18:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    27W /  70W |    805MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**NOTE:** To make it easier for us to manage datasets, images and models we create a `HOME` constant.

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content/GroundingDINO/weights


## Install Grounding DINO & additional dependencies

Grounding DINO is still under heavy development.

In addition to Grounding DINO, we will install some additional packages:
- `roboflow` - To download and send datasets to cloud.
- `dataclasses-json` - To make it easier to process COCO JSON and load data from disk.
- `onemetric` - To compare the results of manual labeling with those obtained with Grounding DINO.

In [ ]:
%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO

# we use latest Grounding DINO model API that is not official yet
!git checkout feature/more_compact_inference_api

!pip install -q -e .
!pip install -q roboflow dataclasses-json onemetric

/content/GroundingDINO/weights
Cloning into 'GroundingDINO'...
remote: Enumerating objects: 401, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 401 (delta 130), reused 121 (delta 121), pack-reused 235
Receiving objects: 100% (401/401), 12.84 MiB | 39.73 MiB/s, done.
Resolving deltas: 100% (206/206), done.
/content/GroundingDINO/weights/GroundingDINO
Branch 'feature/more_compact_inference_api' set up to track remote branch 'feature/more_compact_inference_api' from 'origin'.
Switched to a new branch 'feature/more_compact_inference_api'
  Preparing metadata (setup.py) ... done


**NOTE:** Before we begin, let's still log the version of all the key libraries we will use.

In [ ]:
import torch
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

import roboflow
import supervision

print(
    "roboflow:", roboflow.__version__,
    "; supervision:", supervision.__version__
)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
torch:  2.1 ; cuda:  cu118
roboflow: 1.1.7 ; supervision: 0.4.0


**NOTE:** To run the Grounding DINO model we will need two things: a configuration file and a weights file. The first one is part of the repository we just cloned. The second one we need to download.

In [ ]:
# confirm that configuration file exist

import os

CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(CONFIG_PATH, "; exist:", os.path.isfile(CONFIG_PATH))

/content/GroundingDINO/weights/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py ; exist: True


In [ ]:
# download weights file

%cd {HOME}
!mkdir {HOME}/weights
%cd {HOME}/weights

!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

/content/GroundingDINO/weights
mkdir: cannot create directory ‘/content/GroundingDINO/weights/weights’: File exists
/content/GroundingDINO/weights/weights


In [ ]:
# confirm that weights file exist

import os

WEIGHTS_PATH = os.path.join(HOME, "weights", "groundingdino_swint_ogc.pth")
print(WEIGHTS_PATH, "; exist:", os.path.isfile(WEIGHTS_PATH))

/content/GroundingDINO/weights/weights/groundingdino_swint_ogc.pth ; exist: True


## Load model

In [ ]:
%cd {HOME}/GroundingDINO

from groundingdino.util.inference import Model

model = Model(model_config_path=CONFIG_PATH, model_checkpoint_path=WEIGHTS_PATH)

/content/GroundingDINO/weights/GroundingDINO
final text_encoder_type: bert-base-uncased


In [ ]:
from typing import List

def enhance_class_name(class_names: List[str]) -> List[str]:
    return [
        f"all {class_name}s"
        for class_name
        in class_names
    ]

import cv2

import supervision as sv

## Dataset auto annotation

In [ ]:
import os

def list_image_files(directory: str) -> List[str]:
    image_extensions = [".jpeg", ".jpg", ".png", ".bmp", ".gif"]
    image_files = [f for f in os.listdir(directory) if os.path.splitext(f)[1].lower() in image_extensions]
    return image_files


def save_voc_xml(xml_string: str, file_path: str) -> None:
    with open(file_path, 'w') as f:
        f.write(xml_string)


def image_name_to_xml_name(image_name: str) -> str:
    base_name, _ = os.path.splitext(image_name)
    xml_name = f"{base_name}.xml"
    return xml_name

### Uploading to an empty Roboflow project

In [ ]:
import os

PROJECT_NAME = "Custom-Bench_1"
PROJECT_DESCRIPTION = "Custom-dataset-from-eu-videos-720p"
SOURCE_DIRECTORY_PATH = os.path.join(HOME, "/content/drive/MyDrive/uni_life/PIP/AoM/Data/custom_frames") # Path to new images to be added
CLASSES = ['cars', 'bikes', 'trucks', 'busses', 'scooters', 'motorcycles']
BOX_TRESHOLD = 0.25
TEXT_TRESHOLD = 0.25

In [ ]:
from roboflow import Roboflow

workspace = Roboflow().workspace()
project = workspace.create_project(
    project_name=PROJECT_NAME,
    project_license="MIT",
    project_type="object-detection",
    annotation=PROJECT_DESCRIPTION)

RuntimeError: ignored

### OR uploading to an existing project/dataset:

In [ ]:
from roboflow import Roboflow

# Initialize the Roboflow object with your API key
rf = Roboflow(api_key="...")

# Retrieve your current workspace and project name
print(rf.workspace())

# Specify the project for upload
project = rf.workspace("trafficpulse").project("custombench1")



loading Roboflow workspace...
{
  "name": "trafficpulse",
  "url": "trafficpulse",
  "projects": [
    "trafficpulse/boxbike1",
    "trafficpulse/custombench1",
    "trafficpulse/dino1"
  ]
}
loading Roboflow workspace...
loading Roboflow project...


### Detect and Upload Annotations to New Project

Finally, how about putting Grounding DINO in front of a real test and using it to automatically annotate datasets. Labels will be sent to Roboflow where you can manually review them.

In [40]:
from os import listdir


for image_name in list_image_files(SOURCE_DIRECTORY_PATH):
    image_path = os.path.join(SOURCE_DIRECTORY_PATH, image_name)
    image = cv2.imread(image_path)
    height, width, depth = image.shape
    xml_name = image_name_to_xml_name(image_name=image_name)
    xml_path = os.path.join(SOURCE_DIRECTORY_PATH, xml_name)

    detections = model.predict_with_classes(
        image=image,
        classes=enhance_class_name(class_names=CLASSES),
        box_threshold=BOX_TRESHOLD,
        text_threshold=TEXT_TRESHOLD
    )

    # drop potential detections with phrase that is not part of CLASSES set
    detections = detections[detections.class_id != None]
    # drop potential detections with area close to area of whole image
    detections = detections[(detections.area / (height * width)) < 0.9 ]
    # drop potential double detections
    detections = detections.with_nms()

    xml_string = sv.detections_to_voc_xml(
        detections=detections,
        classes=CLASSES,
        filename=image_name,
        width=width,
        height=height,
        depth=depth
    )

    save_voc_xml(xml_string=xml_string, file_path=xml_path)
    project.upload(image_path=image_path, annotation_path=xml_path, split="train", overwrite=True)

Duplicate image not uploaded: /content/drive/MyDrive/uni_life/PIP/AoM/Data/custom_frames/frame1545.jpg
image already annotated: /content/drive/MyDrive/uni_life/PIP/AoM/Data/custom_frames/frame1545.xml
Duplicate image not uploaded: /content/drive/MyDrive/uni_life/PIP/AoM/Data/custom_frames/frame1546.jpg
image already annotated: /content/drive/MyDrive/uni_life/PIP/AoM/Data/custom_frames/frame1546.xml
Duplicate image not uploaded: /content/drive/MyDrive/uni_life/PIP/AoM/Data/custom_frames/frame1547.jpg
image already annotated: /content/drive/MyDrive/uni_life/PIP/AoM/Data/custom_frames/frame1547.xml
Duplicate image not uploaded: /content/drive/MyDrive/uni_life/PIP/AoM/Data/custom_frames/frame1548.jpg
image already annotated: /content/drive/MyDrive/uni_life/PIP/AoM/Data/custom_frames/frame1548.xml
Duplicate image not uploaded: /content/drive/MyDrive/uni_life/PIP/AoM/Data/custom_frames/frame1549.jpg
image already annotated: /content/drive/MyDrive/uni_life/PIP/AoM/Data/custom_frames/frame1549

/content/drive/MyDrive/uni_life/PIP/AoM/Data/custom_frames/frame1625.xml ERROR saving annotation: save annotation for ikuuIuoiNEi0Qxo8AbBi bad response: <Response [502]>
/content/drive/MyDrive/uni_life/PIP/AoM/Data/custom_frames/frame1746.xml ERROR saving annotation: save annotation for IEFx4r2T9rC6xDk3oCse bad response: <Response [502]>
/content/drive/MyDrive/uni_life/PIP/AoM/Data/custom_frames/frame2048.xml ERROR saving annotation: save annotation for QrxN98w1asaz5Ku8VRkd bad response: <Response [502]>
/content/drive/MyDrive/uni_life/PIP/AoM/Data/custom_frames/frame2200.jpg ERROR uploading image after 0 retries: Bad response: <Response [502]>
/content/drive/MyDrive/uni_life/PIP/AoM/Data/custom_frames/frame2392.xml ERROR saving annotation: save annotation for AkE3OZPDzAA9ciudNF9U bad response: <Response [502]>
/content/drive/MyDrive/uni_life/PIP/AoM/Data/custom_frames/frame2508.xml ERROR saving annotation: save annotation for d9MjZBfIzngYGzWGce1S / bad response: 400: {'message': 'Inva

Duplicate image not uploaded: /content/drive/MyDrive/uni_life/PIP/AoM/Data/custom_frames/frame811.jpg


/content/drive/MyDrive/uni_life/PIP/AoM/Data/custom_frames/frame811.xml ERROR saving annotation: save annotation for EuCtXac6EYUKkbcPWHyO / bad response: 400: {'message': 'Invalid annotations.', 'type': 'InvalidAnnotationException', 'hint': 'All the annotations provided are invalid', 'badAnnotations': [{'type': 'warning', 'code': 'annotation:trimmed', 'key': 'frame811.jpg', 'label': 'cars'}]}
/content/drive/MyDrive/uni_life/PIP/AoM/Data/custom_frames/frame812.xml ERROR saving annotation: save annotation for 6CL12E9c8z7vA9q0UuDb / bad response: 400: {'message': 'Invalid annotations.', 'type': 'InvalidAnnotationException', 'hint': 'All the annotations provided are invalid', 'badAnnotations': [{'type': 'warning', 'code': 'annotation:trimmed', 'key': 'frame812.jpg', 'label': 'cars'}]}
/content/drive/MyDrive/uni_life/PIP/AoM/Data/custom_frames/frame813.xml ERROR saving annotation: save annotation for rV0gKnkPQHitDIibnrm1 / bad response: 400: {'message': 'Invalid annotations.', 'type': 'Inva